In [1]:
import pandas as pd
import polars as pl

In [3]:
ebola = pd.read_csv("data/country_timeseries.csv")
ebola

,Date,Day,Cases_Guinea,Cases_Liberia,Cases_SierraLeone,Cases_Nigeria,Cases_Senegal,Cases_UnitedStates,Cases_Spain,Cases_Mali,Deaths_Guinea,Deaths_Liberia,Deaths_SierraLeone,Deaths_Nigeria,Deaths_Senegal,Deaths_UnitedStates,Deaths_Spain,Deaths_Mali
0,1/5/2015,289,2776.0,NaN,10030.0,NaN,NaN,NaN,NaN,NaN,1786.0,NaN,2977.0,NaN,NaN,NaN,NaN,NaN
1,1/4/2015,288,2775.0,NaN,9780.0,NaN,NaN,NaN,NaN,NaN,1781.0,NaN,2943.0,NaN,NaN,NaN,NaN,NaN
2,1/3/2015,287,2769.0,8166.0,9722.0,NaN,NaN,NaN,NaN,NaN,1767.0,3496.0,2915.0,NaN,NaN,NaN,NaN,NaN
3,1/2/2015,286,NaN,8157.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3496.0,NaN,NaN,NaN,NaN,NaN,NaN
4,12/31/2014,284,2730.0,8115.0,9633.0,NaN,NaN,NaN,NaN,NaN,1739.0,3471.0,2827.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,3/27/2014,5,103.0,8.0,6.0,NaN,NaN,NaN,NaN,NaN,66.0,6.0,5.0,NaN,NaN,NaN,NaN,NaN
118,3/26/2014,4,86.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,62.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
119,3/25/2014,3,86.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
120,3/24/2014,2,86.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
ebola_long = ebola.melt(
    id_vars=["Date", "Day"],
    var_name="cd_country",
    value_name="count",
)
ebola_long

# don't need to know
ebola_long[["case_death", "country"]] = ebola_long.cd_country.str.split(
    "_",
    expand=True,
)
ebola_long

ebola_tidy = (
    ebola_long.drop(columns="cd_country")  # dropping a column
    .pivot(index=["Date", "Day", "country"], columns="case_death", values="count")
    .reset_index()
    .dropna()
)
ebola_tidy

# final clean up of names
# don't need to know
ebola_tidy.columns.name = None
ebola_tidy

,Date,Day,country,Cases,Deaths
1,1/2/2015,286,Liberia,8157.0,3496.0
8,1/3/2015,287,Guinea,2769.0,1767.0
9,1/3/2015,287,Liberia,8166.0,3496.0
13,1/3/2015,287,SierraLeone,9722.0,2915.0
16,1/4/2015,288,Guinea,2775.0,1781.0
...,...,...,...,...,...
960,9/7/2014,169,Guinea,861.0,557.0
961,9/7/2014,169,Liberia,2081.0,1137.0
963,9/7/2014,169,Nigeria,21.0,8.0
964,9/7/2014,169,Senegal,3.0,0.0


In [5]:
ebola_long.groupby(["case_death", "country"])["count"].max().reset_index()

,case_death,country,count
0,Cases,Guinea,2776.0
1,Cases,Liberia,8166.0
2,Cases,Mali,7.0
3,Cases,Nigeria,22.0
4,Cases,Senegal,3.0
5,Cases,SierraLeone,10030.0
6,Cases,Spain,1.0
7,Cases,UnitedStates,4.0
8,Deaths,Guinea,1786.0
9,Deaths,Liberia,3496.0


In [7]:
ebola = pl.read_csv("data/country_timeseries.csv")
ebola

Date,Day,Cases_Guinea,Cases_Liberia,Cases_SierraLeone,Cases_Nigeria,Cases_Senegal,Cases_UnitedStates,Cases_Spain,Cases_Mali,Deaths_Guinea,Deaths_Liberia,Deaths_SierraLeone,Deaths_Nigeria,Deaths_Senegal,Deaths_UnitedStates,Deaths_Spain,Deaths_Mali
str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""1/5/2015""",289,2776,null,10030,null,null,null,null,null,1786,null,2977,null,null,null,null,null
"""1/4/2015""",288,2775,null,9780,null,null,null,null,null,1781,null,2943,null,null,null,null,null
"""1/3/2015""",287,2769,8166,9722,null,null,null,null,null,1767,3496,2915,null,null,null,null,null
"""1/2/2015""",286,null,8157,null,null,null,null,null,null,null,3496,null,null,null,null,null,null
"""12/31/2014""",284,2730,8115,9633,null,null,null,null,null,1739,3471,2827,null,null,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""3/27/2014""",5,103,8,6,null,null,null,null,null,66,6,5,null,null,null,null,null
"""3/26/2014""",4,86,null,null,null,null,null,null,null,62,null,null,null,null,null,null,null
"""3/25/2014""",3,86,null,null,null,null,null,null,null,60,null,null,null,null,null,null,null


In [10]:
ebola_tidy = (
    ebola

    # wide -> long
    .melt(
        id_vars=["Date", "Day"],
        variable_name="cd_country",
        value_name="count",
    )

    # split "Cases_Guinea" / "Deaths_Guinea"
    .with_columns(
        pl.col("cd_country")
        .str.split_exact("_", 1)
        .struct.rename_fields(["case_death", "country"])
        .alias("tmp")
    )
    .unnest("tmp")

    # reshape back to tidy form
    .drop("cd_country")
    .pivot(
        index=["Date", "Day", "country"],
        columns="case_death",
        values="count",
    )
    .drop_nulls()
)

ebola_tidy


C:\Users\Daniel\AppData\Local\Temp\ipykernel_19580\2308573817.py:5: DeprecationWarning: `DataFrame.melt` is deprecated; use `DataFrame.unpivot` instead, with `index` instead of `id_vars` and `on` instead of `value_vars`
  .melt(
C:\Users\Daniel\AppData\Local\Temp\ipykernel_19580\2308573817.py:22: DeprecationWarning: the argument `columns` for `DataFrame.pivot` is deprecated. It was renamed to `on` in version 1.0.0.
  .pivot(


Date,Day,country,Cases,Deaths
str,i64,str,i64,i64
"""1/5/2015""",289,"""Guinea""",2776,1786
"""1/4/2015""",288,"""Guinea""",2775,1781
"""1/3/2015""",287,"""Guinea""",2769,1767
"""12/31/2014""",284,"""Guinea""",2730,1739
"""12/28/2014""",281,"""Guinea""",2706,1708
…,…,…,…,…
"""11/4/2014""",227,"""Mali""",1,1
"""11/2/2014""",225,"""Mali""",1,1
"""10/29/2014""",220,"""Mali""",1,1
